# 📈 ПРОЕКТ 📊

# 🔍 ПАРСИНГ АРЕНДНЫХ ПОМЕЩЕНИЙ С САЙТА 🔎



<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc">
    <ul class="toc-item">
        <li><span><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li>
        <li><span><span class="toc-item-num">2&nbsp;&nbsp;</span>Создание парсера</a></span></li>
        <li><span><span class="toc-item-num">3&nbsp;&nbsp;</span>Вывод таблицы и сохранение в файл</a></span></li>
        <li><span><span class="toc-item-num">4&nbsp;&nbsp;</span>Вывод и рекомендации заказчику</a></span></li>
    </ul>
</div>

# 📝 Описание проекта:  

<div style="border:solid green 3px; padding: 20px">

Заказчик подал запрос на получение информации арендных помещений с сайта Shop'n'Mall. Данный сайт состоит из страниц по 10 объектов на каждой. К каждому объекту, по переходу по ссылке, прикреплены характеристики помещений. Эти характеристики должны быть помещены в таблицу и сохранены в файле формата .scv.


# 📂 Описание данных:

<div style="border:solid green 3px; padding: 20px">

Дан сайт: [ссылка на основной сайт](https://shopandmall.ru/)

Раздел сайта с арендными помещениями: [ссылка](https://shopandmall.ru/offers?page=1&per-page=10)


# 🎯 Цель проекта:

<div style="border:solid green 3px; padding: 20px">

Произвести парсинг раздела арендных помещений сайта Shop'n'Mall (в целях ознакомления в коде проекта будет приведён парсинг объектов первых 10 страниц).

# 📅 План действий:

<div style="border:solid green 3px; padding: 20px">

1. Импортировать библиотеки;
2. Сохранить ссылки;
3. Создать парсер;
4. Вывести таблицу;
5. Сохранить в файл.


## Загрузка данных

<div style="border:solid green 2px; padding: 5px">

импортируем все необходимые библиотеки:

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

<div style="border:solid green 2px; padding: 5px">

сохраним основные ссылки в переменные, на месте указания страниц в динамичной ссылке с разделом арендных помещений поставим {}, в дальнейшем это понадобится, чтобы перебирать страницы до нужной величины:

In [2]:
base_url = 'https://shopandmall.ru'
pages_url = "https://shopandmall.ru/offers?page={}&per-page=10"

<div style="border:solid green 2px; padding: 5px">

в переменную data сохраним все данные, которые парсер вытянет из раздела аренды сайта:

In [3]:
data = []

## Создание парсера

<div style="border:solid green 2px; padding: 5px">

создаём парсер, логика данного парсера такова:
1. Перебираем страницы с 1 по 10;
2. Находим все объекты "catalog-card-info__title-row", в которые вшита ссылка;
3. Определяется наличие ссылки, формируется полноценная ссылка;
4. По ссылке каждого объекта собираются требуемые характеристики;
5. Собранные данные сохраняются в переменную в виде "ключ: значение";
6. Все собранные данные добавляются в переменную data;

In [4]:
for page in range(1, 11):
  url = pages_url.format(page)
  response = requests.get(pages_url)

  if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')

    title_rows = soup.find_all('div', class_='catalog-card-info__title-row')

    for title_row in title_rows:
        link_element = title_row.find('a')

        if link_element and 'href' in link_element.attrs:
            relative_url = link_element['href']
            full_url = base_url + relative_url

            offer_response = requests.get(full_url)

            if offer_response.status_code == 200:
                offer_soup = BeautifulSoup(offer_response.text, 'html.parser')

                characteristics = {}
                content_table = offer_soup.find('div', class_='content-table')

                if content_table:
                    rows = content_table.find_all('div', class_='content-table__row')

                    for row in rows:
                        cols = row.find_all('div', class_='content-table__col')
                        if len(cols) >= 2:
                            key = cols[0].get_text(strip=True)
                            value = cols[1].get_text(strip=True)
                            characteristics[key] = value

                data.append(characteristics)

            else:
                print(f'Ошибка при загрузке страницы предложения: {offer_response.status_code}')

  else:
   print(f'Ошибка при загрузке страницы предложения: {offer_response.status_code}')

## Вывод таблицы и сохранение в файл

<div style="border:solid green 2px; padding: 5px">

Выводим полученные даты в виде таблицы и сохраняем в файл в формате .csv:

In [5]:
rent_table = pd.DataFrame(data)
rent_table.to_csv('rent_table.csv', index=False)

print("Данные успешно сохранены в rent_table.csv")
display(rent_table)

Данные успешно сохранены в rent_table.csv


,№ объявления,Дата размещения,Город,Адрес,Расположено,Этаж,Предлагается,Желаемый / подходящий вид деятельности,Назначение,Размер площади (м2),Цена за помещение,Цена за 1 кв. м
0,111708,07.08.2024,Краснодар,"улица Калинина, 106",Отдельно стоящее здание,2,Аренда,Свободное назначение,"Торговое, Услуги",480,360 000 руб.,750 руб.
1,112282,29.09.2024,Владивосток,"Светланская улица, д.53",,2,Продажа,Не указано,Не указано,294.5,26 421 000 руб.,NaN
2,112283,29.09.2024,Владивосток,"Светланская улица, д.53",,2,Продажа,Не указано,Не указано,1040.5,78 924 000 руб.,NaN
3,112284,29.09.2024,Владивосток,"Светланская улица, д.53",,2,Продажа,Не указано,Не указано,243.6,22 188 000 руб.,NaN
4,112195,29.09.2024,Москва,"Кировоградская улица, д.9к2",Жилой дом,1,Аренда,Не указано,Не указано,67,170 000 руб.,2 538 руб.
...,...,...,...,...,...,...,...,...,...,...,...,...
95,110265,29.09.2024,Москва,"Полковая улица, д.1",Жилой дом,1,Продажа,Не указано,Не указано,56.5,49 451 116 руб.,NaN
96,110267,29.09.2024,Москва,"Полковая улица, д.1",Жилой дом,1,Продажа,Не указано,Не указано,76.4,64 862 377 руб.,NaN
97,112372,29.09.2024,Видное,"деревня Тарычево, Фруктовые сады улица, д.2/10",,-1,Продажа,Не указано,Не указано,5.4,569 700 руб.,NaN
98,112374,29.09.2024,Видное,"деревня Тарычево, Фруктовые сады улица, д.2/10",,-1,Продажа,Не указано,Не указано,3.2,376 000 руб.,NaN


## Вывод и рекомендации заказчику

<div style="border:solid green 4px; padding: 15px">

# Вывод:
    
1. Провели подготовку данных;
    
2. Создали парсер;

3. Выгрузили в виде таблицы и сохранили в файл.

<div style="border-radius: 15px; border: 3px solid indigo; padding: 15px;">

# ЗАКЛЮЧЕНИЕ:

Заказчиком была поставлена задача создать парсер, который выгрузит объекты арендных помещений из специального раздела сайта. Данные должны быть показаны в виде таблицы и сохранены в отдельный файл для последующей обработки. Все задачи были выполнены.